In [22]:
# !/usr/bin/env python
# -*- coding:utf8 -*-
import os
# 常用的中文分词包jieba；相应地，常用的英文分词包是NTLK
import jieba  
# TfidfVectorizer，用来将原始文本转化为tf-idf的特征矩阵
from sklearn.feature_extraction.text import TfidfVectorizer
# 多项式贝叶斯分类器
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics


# 分类标识字典
labels = {'体育': 0, '女性': 1, '文学': 2, '校园': 3}
# 加载停用词
with open('./data/stop/stopword.txt', 'rb') as f:
    STOP_WORDS = [line.strip() for line in f.readlines()]


    
def load_data(path):
#获取分词结果和对应标签

    documents = []
    labels = []    
    # 循环所有文件并进行分词
    # os.walk(path)得到的是多个元组构成的一个生成器，1个元祖的结构是：root表示路径字符串，dirs表示该路径下的文件夹列表，files表示该路径下的文件列表
    for root, dirs, files in os.walk(path): 
        for file in files:
            # 获取标识（体育、女性、文学、校园），注意，windows获取的新路径，斜杆变成了反斜杠。
            label = root.split('\\')[-1] 
            labels.append(label)
            # 路径+文件名合成新的文件路径
            filename = os.path.join(root, file)
            # 因为字符集问题因此直接用二进制方式读取
            with open(filename, 'rb') as f: 
                content = f.read()
                # 使用jieba工具包进行中文分词，得到的结果是字符串列表
                word_list = list(jieba.cut(content))
                # 将子字符串列表，利用join函数生成一个新的长字符串，其中子字符串以空格隔开；
                # 最后循环得到的字符串列表，每一个元素就是一个文档分词的字符串，每个元素中的单词以逗号隔开
                documents.append(' '.join(word_list))
                
    # 返回所有文档的分词列表，以及每个文档一一对应的标识
    return documents, labels


def train_fun(train_data, train_label, test_data, test_label):
    """
    构造模型并计算测试集准确率，字数限制变量名简写
    :param train_data: 训练集数据
    :param train_label: 训练集标签
    :param test_data: 测试集数据
    :param test_label: 测试集标签
    :return: 测试集准确率
    """
    # 计算训练数据集矩阵，max_df=0.5 表示一个单词在50%的文档中都出现过了，因此不作为分词统计。
    tf = TfidfVectorizer(stop_words=STOP_WORDS, max_df=0.5)
    features = tf.fit_transform(train_data)
    # 训练模型
    clf = MultinomialNB(alpha=0.001).fit(features, train_label)
    
    
    # 计算测试数据集矩阵，使用同样的stop_word和max_df，vocabulary来自训练集进行transform_fit拟合模型后的属性
    test_tf = TfidfVectorizer(stop_words=STOP_WORDS, max_df=0.5, vocabulary=tf.vocabulary_)
    test_features = test_tf.fit_transform(test_data)
    # 模型预测
    predicted_labels = clf.predict(test_features)
    # 获取结果
    x = metrics.accuracy_score(test_label, predicted_labels)
    return x


# 获取训练数据集和测试数据集的分词结果和标签
train_documents, train_labels = load_data('./data/train')
test_documents, test_labels = load_data('./data/test')

# 通过训练数据集、测试数据集的分词结果和标签，得到预测结果打分
score = train_fun(train_documents, train_labels, test_documents, test_labels)
print(score)

0.91
